In [ ]:
# Copyright (c) 2020 ZZH

In [ ]:
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras import Model
import os
import numpy as np

In [ ]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
REGULARIZER  = 0.0005
checkpoint_save_path =  './Model/AlexNet/'

In [ ]:
#数据导入及数据增强
cifar10 = tf.keras.datasets.cifar10
(x_train,y_train),(x_test,y_test) = cifar10.load_data()
x_test = x_test/255.0
DataGen = tf.keras.preprocessing.image.ImageDataGenerator(
          rescale = 1./255.0,
          rotation_range = 15,
          width_shift_range = 0.1,
          height_shift_range = 0.1,
          horizontal_flip = True,
          vertical_flip = False,
          shear_range=0.1,
          zoom_range = 0.1)
DataGen.fit(x_train)

In [ ]:
#网络搭建及训练
class AlexNet(Model):
    def __init__(self):
        super(AlexNet,self).__init__()
        self.c1 = Conv2D(filters=96, kernel_size=(3, 3))
        self.b1 = BatchNormalization()
        self.a1 = Activation('relu')
        self.p1 = MaxPool2D(pool_size=(3,3), strides=2)
        
        self.c2 = Conv2D(filters=256, kernel_size=(3, 3))
        self.b2 = BatchNormalization()
        self.a2 = Activation('relu')
        self.p2 = MaxPool2D(pool_size=(3,3), strides=2)   
        
        self.c3 = Conv2D(filters=384, kernel_size=(3, 3), padding='same', activation='relu')
        self.c4 = Conv2D(filters=384, kernel_size=(3, 3), padding='same', activation='relu')
        self.c5 = Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu')
        self.p3 = MaxPool2D(pool_size=(3,3), strides=2)
        
        self.flatten = Flatten()
        self.f1 = Dense(2048,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))
        self.d1 = Dropout(0.5)
        self.f2 = Dense(2048,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))
        self.d2 = Dropout(0.5)
        self.f3 = Dense(10,activation='softmax',kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))
    
    def call(self,x):
        x = self.c1(x)
        x = self.b1(x)
        x = self.a1(x)
        x = self.p1(x)
        
        x = self.c2(x)
        x = self.b2(x)
        x = self.a2(x)
        x = self.p2(x)
        
        x = self.c3(x)
        x = self.c4(x)
        x = self.c5(x)
        x = self.p3(x)
        
        x = self.flatten(x)
        x = self.f1(x)
        x = self.d1(x)
        x = self.f2(x)
        x = self.d2(x)
        y = self.f3(x)
        return y

In [ ]:
model = AlexNet()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.9, epsilon=1e-07, amsgrad=True),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

log_dir = os.path.join("Model","AlexNet_logs")
callbacks = [
            tf.keras.callbacks.ModelCheckpoint(     #模型保存
                filepath = checkpoint_save_path,
                save_weights_only = False,
                monitor = 'val_accuracy',
                save_best_only = True,
                save_freq='epoch'),
            tf.keras.callbacks.EarlyStopping(       #早停
                monitor = 'val_accuracy',
                patience=15, 
                baseline=None),
            tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, update_freq='epoch', write_graph=True, write_images=False)  #保存计算图
            ]

hist = model.fit(DataGen.flow(x_train,y_train,batch_size=256,shuffle=True),
                 epochs=100,
                 validation_data=(x_test,y_test),
                 validation_freq=1,
                 callbacks=callbacks)

model.summary()

In [ ]:
#结果可视化
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use({'figure.figsize':(6,4)})

plt.plot(hist.history['loss'], label='loss')
plt.plot(hist.history['val_loss'], label='val_loss')
plt.legend()
plt.show()
plt.plot(hist.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()

In [ ]:
print('best result: {:.2f}%  ({}epochs)'.format(100*max(hist.history['val_accuracy']),1+hist.history['val_accuracy'].index(max(hist.history['val_accuracy']))))

In [ ]:
#tensorboard可视化
#!tensorboard --logdir=./Model/AlexNet_logs
#http://localhost:6006/